In [2]:
# importing packages and libraries
from mxnet import nd, gluon, autograd
from mxnet.gluon import nn
import mxnet as mx
import numpy as np
import pandas as pd
import time
from mxnet import profiler
import re

In [3]:
data_ctx = mx.gpu()
model_ctx = mx.gpu()


In [4]:
net = nn.Sequential()
net.add(
    # Here, we use a larger 11 x 11 window to capture objects. At the same time,
   # we use a stride of 4 to greatly reduce the height and width of the output.
    nn.Conv2D(96,kernel_size=11,strides=4,activation='relu'),
    nn.MaxPool2D(pool_size=3,strides=2),
    # Make the convolution window smaller, set padding to 2 for consistent
    # height and width across the input and output, and increase the
    # number of output channels
    nn.Conv2D(256,kernel_size=5,padding=2,activation='relu'),
    nn.MaxPool2D(pool_size=3,strides=2),
    # Use three successive convolutional layers and a smaller convolution
    # window. Except for the final convolutional layer, the number of
    # output channels is further increased. Pooling layers are not used to
    # reduce the height and width of input after the first two
    # convolutional layers
   nn.Conv2D(384, kernel_size=3,padding=1,activation='relu'),
   nn.Conv2D(384, kernel_size=3,padding=1,activation='relu'),
   nn.Conv2D(256, kernel_size=3,padding=1,activation='relu'),
   nn.MaxPool2D(pool_size=3, strides=2),
   nn.Dense(4096,activation='relu'),
   nn.Dropout(.5),
   nn.Dense(4096,activation='relu'),
   nn.Dropout(0.5),
   nn.Dense(10)

)

In [5]:
X = np.random.uniform(size=(1,1,224,224))
net.initialize()
for layer in net:
  X=layer(X)
  print(layer.name,'output shape:\t',X.shape)

AssertionError: HybridBlock requires the first argument to forward be either Symbol or NDArray, but got <class 'numpy.ndarray'>

In [5]:
# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements.  See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership.  The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License.  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied.  See the License for the
# specific language governing permissions and limitations
# under the License.

# coding: utf-8
# pylint: disable= arguments-differ
"""Alexnet, implemented in Gluon."""
__all__ = ['AlexNet', 'alexnet']

import os

# from ....context import cpu
from mxnet.context import cpu
from mxnet.gluon.block import HybridBlock
from mxnet.gluon import nn

# Net
class AlexNet(HybridBlock):
    r"""AlexNet model from the `"One weird trick..." `_ paper.

    Parameters
    ----------
    classes : int, default 1000
        Number of classes for the output layer.
    """
    def __init__(self, classes=1000, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
        with self.name_scope():
            self.features = nn.HybridSequential(prefix='')
            with self.features.name_scope():
                self.features.add(nn.Conv2D(64, kernel_size=11, strides=4,
                                            padding=2, activation='relu'))
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))
                self.features.add(nn.Conv2D(192, kernel_size=5, padding=2,
                                            activation='relu'))
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))
                self.features.add(nn.Conv2D(384, kernel_size=3, padding=1,
                                            activation='relu'))
                self.features.add(nn.Conv2D(256, kernel_size=3, padding=1,
                                            activation='relu'))
                self.features.add(nn.Conv2D(256, kernel_size=3, padding=1,
                                            activation='relu'))
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))
                self.features.add(nn.Flatten())
                self.features.add(nn.Dense(4096, activation='relu'))
                self.features.add(nn.Dropout(0.5))
                self.features.add(nn.Dense(4096, activation='relu'))
                self.features.add(nn.Dropout(0.5))

            self.output = nn.Dense(classes)

    def hybrid_forward(self, F, x):
        x = self.features(x)
        x = self.output(x)
        return x


# Constructor
def alexnet(pretrained=False, ctx=cpu(),
            root=os.path.join('~', '.mxnet', 'models'), **kwargs):
    r"""AlexNet model from the `"One weird trick..." `_ paper.

    Parameters
    ----------
    pretrained : bool, default False
        Whether to load the pretrained weights for model.
    ctx : Context, default CPU
        The context in which to load the pretrained weights.
    root : str, default '~/.mxnet/models'
        Location for keeping the model parameters.
    """
    net = AlexNet(**kwargs)
    if pretrained:
        from ..model_store import get_model_file
        net.load_params(get_model_file('alexnet', root=root), ctx=ctx)
    return net


In [1]:
from mxnet import np,npx,init
from mxnet.gluon import nn
import mxnet as d2l
npx.set_np()

In [2]:
net = nn.Sequential()

In [3]:
net.add(
    # Here, we use a larger 11 x 11 window to capture objects. At the same time,
   # we use a stride of 4 to greatly reduce the height and width of the output.
    nn.Conv2D(96,kernel_size=11,strides=4,activation='relu'),
    nn.MaxPool2D(pool_size=3,strides=2),
    # Make the convolution window smaller, set padding to 2 for consistent
    # height and width across the input and output, and increase the
    # number of output channels
    nn.Conv2D(256,kernel_size=5,padding=2,activation='relu'),
    nn.MaxPool2D(pool_size=3,strides=2),
    # Use three successive convolutional layers and a smaller convolution
    # window. Except for the final convolutional layer, the number of
    # output channels is further increased. Pooling layers are not used to
    # reduce the height and width of input after the first two
    # convolutional layers
   nn.Conv2D(384, kernel_size=3,padding=1,activation='relu'),
   nn.Conv2D(384, kernel_size=3,padding=1,activation='relu'),
   nn.Conv2D(256, kernel_size=3,padding=1,activation='relu'),
   nn.MaxPool2D(pool_size=3, strides=2),
   nn.Dense(4096,activation='relu'),
   nn.Dropout(.5),
   nn.Dense(4096,activation='relu'),
   nn.Dropout(0.5),
   nn.Dense(10)

)

In [4]:
X = np.random.uniform(size=(1,1,224,224))
net.initialize()
for layer in net:
  X=layer(X)
  print(layer.name,'output shape:\t',X.shape)

conv0 output shape:	 (1, 96, 54, 54)
pool0 output shape:	 (1, 96, 26, 26)
conv1 output shape:	 (1, 256, 26, 26)
pool1 output shape:	 (1, 256, 12, 12)
conv2 output shape:	 (1, 384, 12, 12)
conv3 output shape:	 (1, 384, 12, 12)
conv4 output shape:	 (1, 256, 12, 12)
pool2 output shape:	 (1, 256, 5, 5)
dense0 output shape:	 (1, 4096)
dropout0 output shape:	 (1, 4096)
dense1 output shape:	 (1, 4096)
dropout1 output shape:	 (1, 4096)
dense2 output shape:	 (1, 10)


In [16]:
batch_size = 128
train_data = d2l.gluon.data.vision.FashionMNIST(train = True, transform = None)
train_iter = d2l.gluon.data.DataLoader(train_data, batch_size, shuffle = True)
test_data = d2l.gluon.data.vision.FashionMNIST(train = False, transform = None)

In [17]:
lr, num_epochs = 0.01,10
d2l.train_ch6(net,train_iter,test_iter,num_epochs,lr,d2l.try_gpu())

AttributeError: module 'mxnet' has no attribute 'train_ch6'

In [ ]:
# Data for prediction
for X,y in test_iter:
  X=X
  y=y
  break

In [ ]:
def get_fashion_mnist_labels(labels):
    text_labels=['t-shirt','trouser','pullover','dress','coat','sandal','shirt','sneaker','bag','ankle boot']
    return text_labels[labels]

In [ ]:
import pandas as pd

In [ ]:
def predict(X,y):
  trl=[]
  prel=[]
  count = 0
  X=X.as_in_ctx(d2l.try_gpu())
  z=net(X).argmax(axis=1)
  n=len(z)
  for i,j in zip(z,y):
    pre=get_fashion_mnist_labels(int(i))
    tru = get_fashion_mnist_labels(int(j))
    if pre==tru:
      count+=1
    trl.append(tru)
    prel.append(pre)
  label = {'true_label':trl,'predicted_label':prel}
  df = pd.DataFrame(label)
  print(df)
  print('The accuracy:',(count/n)*100)

In [ ]:
predict(X[:10],y)